In [ ]:
import spacy 
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
import json
import random

In [26]:
def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        #print(f)
        return(json.load(f))
    
    
def save_data (file, data):
    with open (file, "w", encoding='utf-8') as f:
       data = json.dump(data, f, indent=4)
       #print(data)

def create_training_data(file, type):
    #print(type)
    #print(file)
    data = load_data(file)
    #print(data)
    patterns=[]
    for key, value in data.items():
        #print(key, value)
        for obj in value:
            #print(obj.get("Municipality"))
            pattern = {
                "label":type,
                "pattern": obj.get("Municipality")
                }
            patterns.append(pattern)
        if any(d.get("Municipality")=='Karnes' for d in value): 
            print("found") 
        if not any(d.get("Municipality")=='Karnes' for d in value):
            print('not found')
            if any(d.get("Primary County")=='Karnes' for d in value): 
                print("found in pc")
        #print(item)
        #print(data)
        
        #print(pattern)
        #print(patterns)
    return(patterns)
patterns = create_training_data('../Corridos/data/TexasMunicipalities.json', 'GPE')
#print(patterns)

not found
found in pc


In [ ]:
def test_model(model, text):
    doc = nlp(text)
    results = []
    entities = []
    for ent in doc.ents:
        x=entities.append((ent.start_char, ent.end_char, ent.label_))
        print(x)
        return (results)
'''
    if len(entities) > 0:
        results = [text, {"entities": entities}]
        return (results)
'''
# spacy wants to see training data as below #train_data
#TRAIN_DATA = [(text, {"entities": [(start, end, label)]})]

nlp = spacy.load("tx_ner")
TRAIN_DATA = []
with open ("../Corridos/corrido corpus/ElCorridodeGregorioCortez_X.txt", "r", encoding='utf-8')as f:
    text = f.read()
    #print(text)
    segments = text.split("\n\n")[0:]
    #hits = []
    for segment in segments:
        segment = segment.strip()
        segment = segment.replace("\n", " ")
        results = test_model(nlp, segment)
        if results != None:
            TRAIN_DATA.append(results)

print (len(TRAIN_DATA))
#save_data("../Corridos/data/TxMunicipality_training_data.json", TRAIN_DATA)



In [ ]:
def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        #print(f)
        return(json.load(f))
    
    
def save_data (file, data):
    with open (file, "w", encoding='utf-8') as f:
       data = json.dump(data, f, indent=4)
       #print(data)

def test_model(model, text):
    doc = nlp(text)
    results = []
    entities = []
    for ent in doc.ents:
        x=entities.append((ent.start_char, ent.end_char, ent.label_))
        print(x)
        return (results)